In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
import pickle  # Импортируем 'pickle' для сериализации модели
from sklearn.model_selection import cross_val_score

# Шаг 1: Загрузка данных из CSV файла
input_file_path = r"C:\Users\mila.degtyareva\Desktop\st\kurs\1.csv"
df = pd.read_csv(input_file_path)

# Шаг 2: Применение Label Encoding к категориальным столбцам
# Мы создаем словарь для хранения энкодеров, чтобы декодировать значения при предсказаниях
label_encoders = {}
categorical_columns = ['PROJECT_NAME', 'ISSUE_PRIORITY', 'ISSUE_TYPE']

for column in categorical_columns:
    le = LabelEncoder()
    # Кодуем категориальные столбцы и сохраняем энкодер
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

# Шаг 3: Логарифмическое преобразование столбца 'SUM(STATUS_DURATION_HOUR)'
# Применяется для стабилизации дисперсии и приведения распределения к нормальному виду
df['SUMMA'] = np.log1p(df['SUMMA'])

# Определяем целевую переменную и признаки
X = df[['PROJECT_NAME', 'ISSUE_PRIORITY', 'ISSUE_TYPE']]  # Признаки
y = df['SUMMA']  # Целевая переменная

# Шаг 4: Нормализация признаков
# Создаем и применяем MinMaxScaler
scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(X)

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42, stratify=X['PROJECT_NAME'])

# Вывод размеров обучающей и тестовой выборок
print(f"Размер обучающей выборки: {X_train.shape[0]} строк")
print(f"Размер тестовой выборки: {X_test.shape[0]} строк")

# Проверка пропорций распределения проектов
train_proportions = X_train.round().sum(axis=0) / X_train.shape[0]  # Из-за нормализации точные значения можно округлить
test_proportions = X_test.round().sum(axis=0) / X_test.shape[0]
print("\nПропорции команд в обучающей выборке по нормализированным данным:")
print(train_proportions)
print("\nПропорции команд в тестовой выборке по нормализированным данным:")
print(test_proportions)

# Создание различных моделей для сравнения
models = {
    'Linear Regression': LinearRegression(),
    'Decision Tree': DecisionTreeRegressor(random_state=42),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42)
}

for name, model in models.items():
    # Оценка моделей с помощью кросс-валидации (5 блоков)
    # Используется отрицательное значение MSE, так как `cross_val_score` минимизирует потери
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    cv_scores = -cv_scores  # Превращаем в положительное значение

    # Среднее и стандартное отклонение MSE для кросс-валидации
    mean_cv_score = cv_scores.mean()
    std_cv_score = cv_scores.std()

    print(f'{name} - Cross-Validated Mean Squared Error: {mean_cv_score:.4f} (+/- {std_cv_score:.4f})')

    # Обучение модели на полной обучающей выборке
    model.fit(X_train, y_train)

    # Оценка на обучающей выборке
    y_train_pred = model.predict(X_train)
    mse_train = mean_squared_error(y_train, y_train_pred)
    r2_train = r2_score(y_train, y_train_pred)

    # Оценка на тестовой выборке
    y_test_pred = model.predict(X_test)
    mse_test = mean_squared_error(y_test, y_test_pred)
    r2_test = r2_score(y_test, y_test_pred)

    print(f'{name} - Mean Squared Error на обучающей выборке: {mse_train:.4f}')
    print(f'{name} - R^2 на обучающей выборке: {r2_train:.4f}')
    print(f'{name} - Mean Squared Error на тестовой выборке: {mse_test:.4f}')
    print(f'{name} - R^2 на тестовой выборке: {r2_test:.4f}\n')

# Обучение модели Random Forest для сохранения
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Сохранение обученной модели и энкодеров
with open('random_forest_model.pkl', 'wb') as model_file:
    pickle.dump(rf_model, model_file)

with open('label_encoders.pkl', 'wb') as encoders_file:
    pickle.dump(label_encoders, encoders_file)

# Сохранение скейлера для повторного использования при предсказаниях
with open('scaler.pkl', 'wb') as scaler_file:
    pickle.dump(scaler, scaler_file)

Размер обучающей выборки: 113124 строк
Размер тестовой выборки: 28281 строк

Пропорции команд в обучающей выборке по нормализированным данным:
[0.37751494 0.87516354 0.75992716]

Пропорции команд в тестовой выборке по нормализированным данным:
[0.37753262 0.87224638 0.75930837]
Linear Regression - Cross-Validated Mean Squared Error: 3.3862 (+/- 0.0267)
Linear Regression - Mean Squared Error на обучающей выборке: 3.3860
Linear Regression - R^2 на обучающей выборке: 0.0290
Linear Regression - Mean Squared Error на тестовой выборке: 3.4411
Linear Regression - R^2 на тестовой выборке: 0.0318

Decision Tree - Cross-Validated Mean Squared Error: 2.4288 (+/- 0.0060)
Decision Tree - Mean Squared Error на обучающей выборке: 2.4201
Decision Tree - R^2 на обучающей выборке: 0.3060
Decision Tree - Mean Squared Error на тестовой выборке: 2.4664
Decision Tree - R^2 на тестовой выборке: 0.3061

Random Forest - Cross-Validated Mean Squared Error: 2.4287 (+/- 0.0061)
Random Forest - Mean Squared Error 